# Предсказание зарплтаты по описанию вакансии


Данный проект подготовлен на основании информации о вакансиях, представленной на сайте https://career.habr.com/. В качестве признака будет использоваться описание вакансии, в качестве таргета - зарплата в рублях.

## 1. Парсинг данных
### 1.1 Подготовка

Доступ к веб-станицам позволяет получать модуль `requests` 

In [1]:
import requests      
import numpy as np   
import pandas as pd  
import time         

Для наших исследовательских целей нужно собрать данные по каждой вакнсии с соответствующей ей страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми выложенными вакансиями. Поскольку нас интересует предсказание зарплаты, оставим только те вакансии, где она указана.
Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [2]:
page_link = 'https://career.habr.com/vacancies?type=all&with_salary=true'# 1-я страница 
#https://career.habr.com/vacancies?page=2&type=all&with_salary=1 2-я страница
#https://career.habr.com/vacancies?page=3&type=all&with_salary=1 3-я страница

Воспользуемся генерацией фейкового юзер-агента, например [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [3]:
!pip install fake_useragent

In [4]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [5]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36'

In [6]:
UserAgent().safari

'Mozilla/5.0 (Windows; U; Windows NT 6.1; sv-SE) AppleWebKit/533.19.4 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4'

In [7]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

Мы получили ответ 200, а значит соединение установлено и данные получены.

In [8]:
html = response.content

In [9]:
len(html)

249418

In [10]:
type(html)

bytes

### 1.2 BeautifulSoup

Восползуемся библиотекой BeautifulSoup, которая из сырого и необработанного HTML кода страницы выдаст нам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.


In [11]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [12]:
soup = BeautifulSoup(html, 'html.parser')

In [13]:
type(soup)

bs4.BeautifulSoup

In [14]:
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="ru">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Работа и свежие вакансии для IT специалистов от прямых работодателей — Хабр Карьера
  </title>
  <meta content="Работа и свежие вакансии для IT специалистов в самых разных сферах IT-индустрии: программирование, верстка, дизайн, менеджмент, веб-аналитика, маркетинг и других." name="description"/>
  <meta content="Работа и свежие вакансии для IT специалистов от прямых работодателей — Хабр Карьера" property="og:title"/>
  <meta content="Работа и свежие вакансии для IT специалистов в самых разных сферах IT-индустрии: программирование, верстка, дизайн, менеджмент, веб-аналитика, маркетинг и других." property="og:description"/>
  <meta content="https://career.habr.com/images/career_share.png" property="og:image"/>
  <meta content="Работа и свежие вакансии для IT специалистов от прямых работодателей — Хабр Карьера" name="twitter:title"/>
  <me

Найдем ссылки, которые ведут с главной страницы на вакансии

In [15]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['vacancy-card__title-link'])
obj

<a class="vacancy-card__title-link" href="/vacancies/1000111905">Frontend разработчик</a>

In [16]:
type(obj)

bs4.element.Tag

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём. Вытащим ссылку на эту вакансию. Сделать это можно по атрибуту `href`, в котором лежит наша ссылка. 

In [17]:
obj.attrs['href']

'/vacancies/1000111905'

После всех преобразований у данных поменялся тип. Теперь они `str`. Это означет, что с ними можно работать как с текстом и пускать в ход для отсеивания лишней информации регулярные выражения. 

In [18]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Тип данных до вытаскивания ссылки: <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки: <class 'str'>


Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с вакансиями.

In [19]:
vacansy_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['vacancy-card__title-link'])
vacansy_links[:3]

[<a class="vacancy-card__title-link" href="/vacancies/1000111905">Frontend разработчик</a>,
 <a class="vacancy-card__title-link" href="/vacancies/1000111910">SRE (DevOps) Kubernetes</a>,
 <a class="vacancy-card__title-link" href="/vacancies/1000080748">Java-разработчик</a>]

In [20]:
len(vacansy_links)

25

Осталось очистить полученный список от мусора:

In [21]:
vacansy_links = [link.attrs['href'] for link in vacansy_links]

In [22]:
vacansy_links[:10]

['/vacancies/1000111905',
 '/vacancies/1000111910',
 '/vacancies/1000080748',
 '/vacancies/1000111909',
 '/vacancies/1000111288',
 '/vacancies/1000100170',
 '/vacancies/1000056761',
 '/vacancies/1000111509',
 '/vacancies/1000112368',
 '/vacancies/1000111275']

In [23]:
len(vacansy_links)

25

Мы получили ровно 25 ссылок по числу вакансий на одной странице поиска. 
Остался последний момент. Когда мы скачаем все вакансии с текущей страницы, нам нужно будет перейти на следующую. 

Обернем в функцию все преобразования, проделанные выше:

In [24]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы
        
        page_number: int/string
            номер страницы для парсинга
            
    """
    # составляем ссылку на страницу поиска
    page_link = 'https://career.habr.com/vacancies?page={}&type=all&with_salary=1'.format(page_number)
    
    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    vacansy_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['vacancy-card__title-link'])
    vacansy_links = ['https://career.habr.com' + link.attrs['href'] for link in vacansy_links]
    
    return vacansy_links

Протестируем функцию и убедимся, что всё хорошо

In [25]:
vacansy_links = getPageLinks(1)
vacansy_links[:2]

['https://career.habr.com/vacancies/1000111905',
 'https://career.habr.com/vacancies/1000111910']

In [26]:
vacansy_links = getPageLinks(2)
vacansy_links[:2]

['https://career.habr.com/vacancies/1000084899',
 'https://career.habr.com/vacancies/1000101124']

Отлично, функция работает и теперь мы теоретически можем достать ссылки на все вакансии, для чего нам придется пройтись по 27 страницам. Перед этим посмотрим, как доставать всю необходимую информацию о конкретной вакансии. 

### 1.3 Сбор информации с каждой страницы 

Для начала сохраним ссылку на страницу в переменную и вытащим по ней контент.

In [27]:
vacancy_page = 'https://career.habr.com/vacancies/1000070949'

response = requests.get(vacancy_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

In [28]:
soup

<!DOCTYPE html>
<html lang="ru"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><title>Вакансия «Middle QA engineer », удаленно, работа в компании «BnBerry» — Хабр Карьера</title>
<meta content="Вакансия «Middle QA engineer », удаленно, работа в компании «BnBerry». Полная занятость. Можно удаленно." name="description"/>
<meta content="Вакансия «Middle QA engineer », удаленно, работа в компании «BnBerry» — Хабр Карьера" property="og:title"/>
<meta content="Вакансия «Middle QA engineer », удаленно, работа в компании «BnBerry». Полная занятость. Можно удаленно." property="og:description"/>
<meta content="https://habrastorage.org/getpro/moikrug/uploads/company/100/007/028/1/logo/medium_ecb79f4255fb4aa8fe1e03011b6d6055.png" property="og:image"/>
<meta content="Вакансия «Middle QA engineer », удаленно, работа в компании «BnBerry» — Хабр Карьера" name="twitter:title"/>
<meta content="Вакансия «Middle QA engineer », удаленно, работа в компании 

найдем информацию о названии вакансии

In [29]:
vacancy = soup.find('h1', attrs={'class':'page-title__title'})
vacancy

<h1 class="page-title__title">Middle QA engineer </h1>

In [30]:
type(vacancy)

bs4.element.Tag

In [31]:
vacancy.text

'Middle QA engineer '

возьием информацию о нашем целевом признаке - зарплате

In [32]:
salary = soup.find('div', attrs={'class':'basic-salary--appearance-vacancy-header'})
salary 

<div class="basic-salary basic-salary--appearance-vacancy-header">до 2000 $</div>

In [33]:
salary = salary.text

In [34]:
salary

'до 2000 $'

перейдем к описанию вакансии, в данном проекте это будет основным признаком, на основании которого мы будем строить наше предсказание

In [62]:
requirements = soup.find('div', attrs={'class':'style-ugc'})
requirements

<div class="style-ugc"><p>Американская компания BnBerry ищет инженера по контролю качества среднего уровня для работы над передовым проектом бронирования отелей.</p>
<p>Мы создаем инновационные продукты в индустрии туристических технологий, которые помогают отелям подключать новые каналы продаж, общаться с гостями и управлять контентом. Среди наших клиентов - крупные международные отели и курорты, а также независимые отели в США. В 2022 году мы выходим на рынки Латинской Америки, Канады и Европы.</p>
<p>Инструменты и сервисы BnBerry используются отелями для подключения к гостиничным рынкам, управления распределением запасов, бронированиями, доходами и обслуживанием клиентов. В нашем портфолио представлены лучшие международные отели и курорты, а также независимые и франчайзинговые отели.</p>
<p><strong>Основные обязанности</strong></p>
<p>Помощь в определении методологии тестирования, критерий и тестовых примеров, необходимых для проверки критической функциональности</p>
<p>Выявление де

In [63]:
for br in requirements('br'):
    br.replace_with('\n')

In [64]:
type(requirements)

bs4.element.Tag

In [65]:
#requirements.br.extend(" ")

In [66]:
requirements

<div class="style-ugc"><p>Американская компания BnBerry ищет инженера по контролю качества среднего уровня для работы над передовым проектом бронирования отелей.</p>
<p>Мы создаем инновационные продукты в индустрии туристических технологий, которые помогают отелям подключать новые каналы продаж, общаться с гостями и управлять контентом. Среди наших клиентов - крупные международные отели и курорты, а также независимые отели в США. В 2022 году мы выходим на рынки Латинской Америки, Канады и Европы.</p>
<p>Инструменты и сервисы BnBerry используются отелями для подключения к гостиничным рынкам, управления распределением запасов, бронированиями, доходами и обслуживанием клиентов. В нашем портфолио представлены лучшие международные отели и курорты, а также независимые и франчайзинговые отели.</p>
<p><strong>Основные обязанности</strong></p>
<p>Помощь в определении методологии тестирования, критерий и тестовых примеров, необходимых для проверки критической функциональности</p>
<p>Выявление де

In [67]:
requirements = requirements.text.replace('\n', ' ')

In [68]:
requirements

'Американская компания BnBerry ищет инженера по контролю качества среднего уровня для работы над передовым проектом бронирования отелей. Мы создаем инновационные продукты в индустрии туристических технологий, которые помогают отелям подключать новые каналы продаж, общаться с гостями и управлять контентом. Среди наших клиентов - крупные международные отели и курорты, а также независимые отели в США. В 2022 году мы выходим на рынки Латинской Америки, Канады и Европы. Инструменты и сервисы BnBerry используются отелями для подключения к гостиничным рынкам, управления распределением запасов, бронированиями, доходами и обслуживанием клиентов. В нашем портфолио представлены лучшие международные отели и курорты, а также независимые и франчайзинговые отели. Основные обязанности Помощь в определении методологии тестирования, критерий и тестовых примеров, необходимых для проверки критической функциональности Выявление дефектов и написание четких и воспроизводимых отчетов об ошибках в соответствии

также добавим информацию о компании

In [69]:
company = soup.find('div', attrs={'class':'vacancy-company__title'})
company

<div class="vacancy-company__title"><a class="basic-avatar basic-avatar--size-extra-small basic-avatar--roundness-small" href="/companies/bnberry"><img alt="Логотип компании «BnBerry»" class="basic-avatar__image" src="https://habrastorage.org/getpro/moikrug/uploads/company/100/007/028/1/logo/ecb79f4255fb4aa8fe1e03011b6d6055.png"/></a><a class="link-comp link-comp--appearance-dark" href="/companies/bnberry">BnBerry</a></div>

In [70]:
company = company.text

In [71]:
company

'BnBerry'

объединим все в общую функцию

In [73]:
def getVacancyData(vacancy_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными
        
        vacancy_page: string
            ссылка на страницу с вакансией
    
    """
    
    # запрашиваем данные по ссылке
    response = requests.get(vacancy_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    vacancy = (soup.find('h1', attrs={'class':'page-title__title'})).text
    salary = (soup.find('div', attrs={'class':'basic-salary--appearance-vacancy-header'})).text
    requirements = (soup.find('div', attrs={'class':'style-ugc'}))
    for br in requirements('br'):
        br.replace_with('\n')
    requirements = requirements.text.replace('\n', ' ')
    company = (soup.find('div', attrs={'class':'vacancy-company__title'})).text

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"vacancy":vacancy, "salary":salary, 
                "requirements":requirements, "company":company}


    return data_row

In [74]:
data_row = getVacancyData('https://career.habr.com/vacancies/1000070949')

In [75]:
data_row

{'vacancy': 'Middle QA engineer ',
 'salary': 'до 2000 $',
 'requirements': 'Американская компания BnBerry ищет инженера по контролю качества среднего уровня для работы над передовым проектом бронирования отелей. Мы создаем инновационные продукты в индустрии туристических технологий, которые помогают отелям подключать новые каналы продаж, общаться с гостями и управлять контентом. Среди наших клиентов - крупные международные отели и курорты, а также независимые отели в США. В 2022 году мы выходим на рынки Латинской Америки, Канады и Европы. Инструменты и сервисы BnBerry используются отелями для подключения к гостиничным рынкам, управления распределением запасов, бронированиями, доходами и обслуживанием клиентов. В нашем портфолио представлены лучшие международные отели и курорты, а также независимые и франчайзинговые отели. Основные обязанности Помощь в определении методологии тестирования, критерий и тестовых примеров, необходимых для проверки критической функциональности Выявление деф

подготовим датафрейм, куда будем добавлять всю полученную информацию

In [76]:
final_df = pd.DataFrame(columns=['vacancy', 'salary', 'requirements', 'company'])

In [77]:
final_df = final_df.append(data_row, ignore_index=True)

<ipython-input-77-d7ed244fc475>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)


In [78]:
final_df

,vacancy,salary,requirements,company
0,Middle QA engineer,до 2000 $,Американская компания BnBerry ищет инженера по...,BnBerry


Еще раз убедимся что всё работает — пройдемся по списку из ссылок на вакансии, полученных ранее в перменной `vacancy_links`.

In [79]:
from tqdm import tqdm_notebook

In [80]:
vacansy_links

['https://career.habr.com/vacancies/1000084899',
 'https://career.habr.com/vacancies/1000101124',
 'https://career.habr.com/vacancies/1000053016',
 'https://career.habr.com/vacancies/1000077160',
 'https://career.habr.com/vacancies/1000060050',
 'https://career.habr.com/vacancies/1000111329',
 'https://career.habr.com/vacancies/1000111326',
 'https://career.habr.com/vacancies/1000112338',
 'https://career.habr.com/vacancies/1000111866',
 'https://career.habr.com/vacancies/1000111865',
 'https://career.habr.com/vacancies/1000111862',
 'https://career.habr.com/vacancies/1000111475',
 'https://career.habr.com/vacancies/1000107505',
 'https://career.habr.com/vacancies/1000098844',
 'https://career.habr.com/vacancies/1000108401',
 'https://career.habr.com/vacancies/1000111851',
 'https://career.habr.com/vacancies/1000108874',
 'https://career.habr.com/vacancies/1000112331',
 'https://career.habr.com/vacancies/1000112330',
 'https://career.habr.com/vacancies/1000110778',
 'https://career.hab

In [81]:
for vacansy_link in tqdm_notebook(vacansy_links):
    try: 
        data_row = getVacancyData(vacansy_link)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.4)
    except:
        continue

<ipython-input-81-5387d6e42254>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for vacansy_link in tqdm_notebook(vacansy_links):


  0%|          | 0/25 [00:00<?, ?it/s]

<ipython-input-81-5387d6e42254>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-81-5387d6e42254>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-81-5387d6e42254>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-81-5387d6e42254>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-81-5387d6e42254>:4: FutureWarning: The frame.append method is deprecated and will

In [82]:
final_df.drop_duplicates(inplace=True)

In [83]:
final_df.shape

(26, 4)

In [84]:
final_df.head()

,vacancy,salary,requirements,company
0,Middle QA engineer,до 2000 $,Американская компания BnBerry ищет инженера по...,BnBerry
1,Fullstack Node.js / Vue.js Developer 🔥,от 2500 до 4500 $,Ищем Fullstack Node.js/Vue.js Developer (верст...,Fundraise Up
2,Data Analyst (middle/senior) (удаленно) 🔥,от 2500 до 3500 $,Ищем дата-аналитика с 3+ годами опыта на удале...,Fundraise Up
3,Senior Frontend-разработчик (React),от 200 000 ₽,"Мы DNA Team — команда, которая предоставляет у...",DNA Team
4,Программист Oracle + Delphi,от 190 000 ₽,Мы сейчас ищем Старших/ Ведущих разработчиков ...,Sportmaster Lab


In [85]:
from tqdm import tqdm_notebook


final_df = pd.DataFrame(columns=['vacancy', 'salary', 'requirements', 'company'])

for page_number in tqdm_notebook(range(29), desc='Pages'):
    vacancy_links = getPageLinks(page_number)  
    for vacancy_link in tqdm_notebook(vacancy_links, leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(3):
            try:
                # пытаемся собрать по мему немного даты
                data_row = getVacancyData(vacancy_link)           
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)  
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', vacancy_link)
                continue

<ipython-input-85-8708ee457813>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page_number in tqdm_notebook(range(29), desc='Pages'):


Pages:   0%|          | 0/29 [00:00<?, ?it/s]

<ipython-input-85-8708ee457813>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for vacancy_link in tqdm_notebook(vacancy_links, leave=False):


  0%|          | 0/25 [00:00<?, ?it/s]

<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and

...

<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and

  0%|          | 0/25 [00:00<?, ?it/s]

<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and

  0%|          | 0/25 [00:00<?, ?it/s]

<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
<ipython-input-85-8708ee457813>:15: FutureWarning: The frame.append method is deprecated and

In [86]:
final_df.head(10)

,vacancy,salary,requirements,company
0,PHP разработчик (Senior),от 300 000 ₽,"Профессиональная команда, успешные проекты в с...",ZeroX
1,Senior Vue Developer,от 250 000 ₽,ZeroX - Мы занимаемся созданием развлекательны...,ZeroX
2,Smart Contract Developer/Solidity Developer (R...,от 3000 $,HashEx vision is to become the #1 DeFi Intelli...,HashEx
3,Frontend разработчик,от 100 000 до 150 000 ₽,"О нас По сути мы стартап, который совсем неда...",Almak IT
4,SRE (DevOps) Kubernetes,от 300 000 ₽,Мониторинг TerraSight — сервис для мониторинга...,Сбер
5,Java-разработчик,от 200 000 до 400 000 ₽,Группы разработки Sportmaster Lab ищут в свою...,Sportmaster Lab
6,Технический менеджер в сервис Compute,от 300 000 ₽,Облачная платформа SberInfra.Cloud является фу...,Сбер
7,Frontend-разработчик (Vue.js),от 80 000 до 140 000 ₽,Требуемый опыт работы: 1–3 года Факт — аккреди...,Факт
8,Инженер технической поддержки,от 120 000 до 140 000 ₽,"Компания ""АйТи Бастион""- российский разработчи...",АйТи Бастион
9,Fullstack / Backend разработчик PHP/JS,от 100 000 ₽,Кто мы? YOURNEEDS - это команда профессионало...,YOURNEEDS


In [87]:
len(final_df)

725

Сохраним полученные данные в csv

In [88]:
final_df.to_csv("habr_parsing2.csv")

In [84]:
final_df.to_csv("habr_parsing_1.csv")

## 2. Предобработка данных

### 2.1 Предобработка целевого признака

In [3]:
final_df = pd.read_csv('habr_parsing2.csv')

In [4]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    725 non-null    int64 
 1   vacancy       725 non-null    object
 2   salary        725 non-null    object
 3   requirements  725 non-null    object
 4   company       725 non-null    object
dtypes: int64(1), object(4)
memory usage: 28.4+ KB


In [5]:
final_df.head(50)

,Unnamed: 0,vacancy,salary,requirements,company
0,0,PHP разработчик (Senior),от 300 000 ₽,"Профессиональная команда, успешные проекты в с...",ZeroX
1,1,Senior Vue Developer,от 250 000 ₽,ZeroX - Мы занимаемся созданием развлекательны...,ZeroX
2,2,Smart Contract Developer/Solidity Developer (R...,от 3000 $,HashEx vision is to become the #1 DeFi Intelli...,HashEx
3,3,Frontend разработчик,от 100 000 до 150 000 ₽,"О нас По сути мы стартап, который совсем неда...",Almak IT
4,4,SRE (DevOps) Kubernetes,от 300 000 ₽,Мониторинг TerraSight — сервис для мониторинга...,Сбер
5,5,Java-разработчик,от 200 000 до 400 000 ₽,Группы разработки Sportmaster Lab ищут в свою...,Sportmaster Lab
6,6,Технический менеджер в сервис Compute,от 300 000 ₽,Облачная платформа SberInfra.Cloud является фу...,Сбер
7,7,Frontend-разработчик (Vue.js),от 80 000 до 140 000 ₽,Требуемый опыт работы: 1–3 года Факт — аккреди...,Факт
8,8,Инженер технической поддержки,от 120 000 до 140 000 ₽,"Компания ""АйТи Бастион""- российский разработчи...",АйТи Бастион
9,9,Fullstack / Backend разработчик PHP/JS,от 100 000 ₽,Кто мы? YOURNEEDS - это команда профессионало...,YOURNEEDS


Запралату указана как в рублях, так и в долларах. Приведем ее к рублям. Официальный курс на сегодня 
(11.10.2022) - 62,31. Кроме того воспользуемся регулярными выражениями, чтобы оставить только число. Если для зарплаты указан диапазон, возьмем только левую границу.

In [6]:
import re

In [7]:
#напишем функцию, которая переводит доллары в рубли и оставляет только численные значения
def total_salary(row):
    row['salary'] = row['salary'].split(' ', maxsplit = 1)[1]
    if '$' in row['salary']:
        return round(int(re.sub(r'[^\d/]', '', row['salary'].split('до')[0]))*62.31)
    else:
        return int(re.sub(r'[^\d/]', '', row['salary'].split('до')[0]))

In [8]:
final_df['salary'] = final_df.apply(total_salary, axis = 1)
final_df.head(20)

,Unnamed: 0,vacancy,salary,requirements,company
0,0,PHP разработчик (Senior),300000,"Профессиональная команда, успешные проекты в с...",ZeroX
1,1,Senior Vue Developer,250000,ZeroX - Мы занимаемся созданием развлекательны...,ZeroX
2,2,Smart Contract Developer/Solidity Developer (R...,186930,HashEx vision is to become the #1 DeFi Intelli...,HashEx
3,3,Frontend разработчик,100000,"О нас По сути мы стартап, который совсем неда...",Almak IT
4,4,SRE (DevOps) Kubernetes,300000,Мониторинг TerraSight — сервис для мониторинга...,Сбер
5,5,Java-разработчик,200000,Группы разработки Sportmaster Lab ищут в свою...,Sportmaster Lab
6,6,Технический менеджер в сервис Compute,300000,Облачная платформа SberInfra.Cloud является фу...,Сбер
7,7,Frontend-разработчик (Vue.js),80000,Требуемый опыт работы: 1–3 года Факт — аккреди...,Факт
8,8,Инженер технической поддержки,120000,"Компания ""АйТи Бастион""- российский разработчи...",АйТи Бастион
9,9,Fullstack / Backend разработчик PHP/JS,100000,Кто мы? YOURNEEDS - это команда профессионало...,YOURNEEDS


In [12]:
final_df['salary'].unique().max()

498480

### 2.2 Предобработка текста

В данной работе мы будем предсказывать зарплату на основании описания вакансии - requirements. Для этого предобработаем полученный текст

Для начала разобьем текст на токены - слова и удалим из него стопслова

In [93]:
import string # библиотека для работы со строками
import nltk   # Natural Language Toolkit

In [94]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:
# знаки препинания
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Инициализируем WordPunctTokenizer, с помощью которого затем разобьем текст на слова

In [96]:
word_tokenizer = nltk.WordPunctTokenizer()

In [97]:
data = final_df[['salary', 'requirements']]

Запишем предобработку текста в виде функции.

In [98]:
def process_data(row):
    text_lower = row['requirements'].lower() # приводим все слова к нижнему регистру
    tokens = word_tokenizer.tokenize(text_lower) #разбиваем текст на слова
        
    # удаляем пунктуацию и стоп-слова
    tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and word not in '—')]
    return tokens
        

In [99]:
data['requirements'] = data.apply(process_data, axis = 1)

<ipython-input-99-7be291111b96>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['requirements'] = data.apply(process_data, axis = 1)


In [100]:
data.head(5)

,salary,requirements
0,300000,"[профессиональная, команда, успешные, проекты,..."
1,250000,"[zerox, занимаемся, созданием, развлекательных..."
2,186930,"[hashex, vision, is, to, become, the, 1, defi,..."
3,100000,"[сути, стартап, который, недавно, образовался,..."
4,300000,"[мониторинг, terrasight, сервис, мониторинга, ..."


In [202]:
!pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=9f5c8c92dcc3d40c4b7fcaf63ee5fadeefd650403a26d3cfbde1476ccb67114f
  Stored in directory: c:\users\lenin\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [101]:
# загружаем библиотеку для лемматизации
import pymorphy2 # Морфологический анализатор

# инициализируем лемматизатор :)
morph = pymorphy2.MorphAnalyzer()

In [102]:
yy = data['requirements'][1]
yy

['zerox',
 'занимаемся',
 'созданием',
 'развлекательных',
 'платформ',
 'международном',
 'рынке',
 'портфеле',
 '10',
 'развитых',
 'работающих',
 'проектов',
 'россии',
 'снг',
 'профессиональная',
 'команда',
 'успешные',
 'проекты',
 'сфере',
 'геи',
 '̆',
 'минга',
 'гэмблинг',
 'индустрии',
 'имеем',
 '10',
 'летнии',
 '̆',
 'опыт',
 'работы',
 'игорнои',
 '̆',
 'индустрии',
 'сеи',
 '̆',
 'час',
 'ищем',
 'сильного',
 'senior',
 'frontend',
 'vue',
 'новою',
 'команду',
 'проект',
 'которыи',
 '̆',
 'содержит',
 'legacy',
 'кода',
 'обязанности',
 'разработка',
 'архитектуры',
 'приложениянепосредственное',
 'участие',
 'создании',
 'проектакод',
 'ревью',
 'остальных',
 'разработчиков',
 'фронтендапоиск',
 'оптимальных',
 'наилучших',
 'решении',
 '̆',
 'участие',
 'планирование',
 'требования',
 'отличное',
 'знание',
 'фраи',
 '̆',
 'мворков',
 'vue',
 'js',
 '2',
 '3умение',
 'работы',
 'websocketхорошее',
 'знание',
 'docker',
 'плюсом',
 'знание',
 'backend',
 'laravel',


In [103]:
xx = ' '.join([morph.parse(x)[0].normal_form for x in yy])

In [104]:
xx

'zerox заниматься создание развлекательный платформа международный рынок портфель 10 развитой работать проект россия снг профессиональный команда успешный проект сфера гей ̆ минг гэмблинга индустрия иметь 10 летнии ̆ опыт работа игорной ̆ индустрия сеи ̆ час искать сильный senior frontend vue новый команда проект который ̆ содержать legacy код обязанность разработка архитектура приложениянепосредственный участие создание проектакод ревить остальной разработчик фронтендапоиск оптимальный хороший решение ̆ участие планирование требование отличный знание фрая ̆ мворк vue js 2 3умение работа websocketхорошеий знание docker плюс знание backend laravel symfony знание nodejs pythonзнание англия ̆ ский использовать php 7 4 laravel 8 9 node js mysql postgresql vue javascript typescript html css3 figma gitlab kubernetes apache kafka условие работать условие тк рфгибкия ̆ график работа обязательный время 12 18 час ), главноерезультат'

In [241]:
#data['requirements'] = ' '.join([morph.parse(x)[0].normal_form for x in data['requirements']])

<ipython-input-241-474ba84d507d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['requirements'] = ' '.join([morph.parse(x)[0].normal_form for x in data['requirements']])


In [248]:
data.head(3)

,salary,requirements
0,249240,работа лондонской prop tech компании команде 1...
1,120000,привет компания html academy школа профессиона...
2,160000,наша компания занимается разработкой приборов ...


In [105]:
def lemmatized(row):
    lem_row = ' '.join([morph.parse(x)[0].normal_form for x in row['requirements']])
    return lem_row

In [106]:
data['requirements'] = data.apply(lemmatized, axis = 1)

<ipython-input-106-6ef2352ec966>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['requirements'] = data.apply(lemmatized, axis = 1)


In [107]:
data.head(3)

,salary,requirements
0,300000,профессиональный команда успешный проект сфера...
1,250000,zerox заниматься создание развлекательный плат...
2,186930,hashex vision is to become the 1 defi intellig...


In [108]:
y = data['salary']
texts = data['requirements']

Отложим часть данных для тестирования и оценки качества алгоритма. Для этого воспользуемся функцией train_test_split

In [109]:
#train test_split
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_y, test_y = train_test_split(texts, y, test_size=0.2, random_state=42)

Воспользуемся моделью Bag of Words, которая реализована в библиотеке sklearn в классе feature_extraction.text.CountVectorizer.

In [110]:
#Инициализируем векторайзер
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 100)
vectorizer.fit(train_texts)

# Топ-10 слов
vectorizer.get_feature_names()[:10]

['1с', 'and', 'api', 'docker', 'git', 'in', 'it', 'js', 'of', 'php']

Обучаем vectorizer на train-данных и сразу преобразем их в вектора с помощью метода fit_transform.

In [111]:
# Обучаем vectorizer на train-данных и сразу преобразем их в вектора с помощью метода fit_transform
train_X = vectorizer.fit_transform(train_texts)
train_X.todense()[:2]

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
         0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 1, 1, 0, 1, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
         2, 2, 0, 2, 4, 0, 0, 0, 0, 3, 1, 1, 1, 0, 0, 1, 0, 3, 1, 0, 1,
         0, 0, 1, 0, 1, 2, 0, 0, 2, 1, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 1, 0, 1, 0, 0, 5, 1, 0,
         1, 0, 0, 0, 2, 0, 1, 0, 1, 5, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 1,
         2, 0, 1, 1, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 4, 0, 0, 2, 1,
         0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 2, 2, 0]], dtype=int64)

Также применяем обученный vectorizer к данным для тестирования.

In [112]:
test_X  = vectorizer.transform(test_texts)

## 3. Обучение модели

### 3.1 Bag of words

Воспользуемся моделью линейной регрессии для предсказаний. В качестве метрики будем использовать RMSE, чтобы значение ошибки имело размерность исходных данных и было более интерпретируемо.

In [113]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [114]:
model = LinearRegression()
model.fit(train_X, train_y)

LinearRegression()

проверим качество модели на кроссвалидации

In [115]:
score = cross_val_score(model, train_X, train_y, scoring = 'neg_mean_squared_error', cv=5)
rmse = np.mean(np.sqrt(np.abs(score)))
print('RMSE: {:.2f}'.format(rmse))

RMSE: 86003.21


и на отложенной тестовой выборке:

In [116]:
pred = model.predict(test_X)

In [117]:
rmse = (np.sqrt(mean_squared_error(test_y, pred)))
print('RMSE: {:.2f}'.format(rmse))

RMSE: 76939.00


Воспользуемся моделью TF-IDF, которая реализована в библиотеке sklearn в классе feature_extraction.TfidfVectorizer

In [125]:
#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# Fit TF-IDF on train texts
vectorizer = TfidfVectorizer(max_features = 100, norm = None) # возмем топ 200 слов
vectorizer.fit(train_texts)

# Топ-10 слов
vectorizer.get_feature_names()[:10]

['1с', 'and', 'api', 'docker', 'git', 'in', 'it', 'js', 'of', 'php']

In [126]:
# Обучаем TF-IDF на train, а затем применяем к train и test
train_X = vectorizer.fit_transform(train_texts)
test_X  = vectorizer.transform(test_texts)

In [127]:
# Пример
train_X.todense()[:2] # посмотрим на первые 2 строки

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 2.48955343, 0.        , 0.        , 0.        ,
         2.19996478, 2.31489475, 0.        , 0.        , 2.56072971,
         0.        , 0.        , 1.63140948, 0.        , 0.        ,
         0.        , 0.        , 2.56072971, 2.26488433, 1.53875065,
         0.        , 0.        , 0.        , 2.58562726, 0.        ,
         2.43749707, 0.        , 2.37431817, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 2.22895232,
         0.        , 0.        , 1.70526854, 2.63736294, 0.        ,
         2.29584655, 2.3653317 , 2.11772668, 0.        , 0.        ,
         2.11247733, 2.38801701, 0.        , 0.        , 0.        ,
         0.        , 0.        , 1.83929782, 3.0482182 , 4.94880326,
         0.        , 3.87081026, 4.51352467, 0.        , 0.        ,
         0.        , 0.        , 3

In [128]:
model = LinearRegression()
model.fit(train_X, train_y)

LinearRegression()

In [129]:
score = cross_val_score(model, train_X, train_y, scoring = 'neg_mean_squared_error', cv=5)
rmse = np.mean(np.sqrt(np.abs(score)))
print('RMSE: {:.2f}'.format(rmse))

RMSE: 86003.21


Посмотрим на качество модели на тестовых данных

In [130]:
pred = model.predict(test_X)

In [131]:
rmse = (np.sqrt(mean_squared_error(test_y, pred)))
print('RMSE: {:.2f}'.format(rmse))

RMSE: 76938.99


**Вывод** Ошибка в определнии зарплаты на основании описания вакансии оказалась достаточно высокой. В дальнейшем данное исследования планируется продолжить для выпускного проекта. Для повышения качества будет использован увеличенный датасет (данные будут спарсены с hh, где количество вакансий гораздо больше), будут использованы дополнительные признаки: местоположение и название компании, наименование вакансии, а также более сложные трансформерные модели.